In [1]:
!pip install datasets
from datasets import load_dataset

data_files = {
    "train": "drive/MyDrive/Colab Notebooks/dataset/nlp/indosum/test.04.jsonl",
}

# Muat dataset
dataset = load_dataset("json", data_files=data_files)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary'],
        num_rows: 3752
    })
})


In [2]:
train_data = dataset['train']
print(train_data[0])

{'category': 'tajuk utama', 'gold_labels': [[True], [False, False, False], [False], [True], [True], [False], [False], [False], [False, False], [False], [False, False, False], [False, False], [False]], 'id': '1504151262-wiranto-yakin-uang-bos-first-travel-bisa-ditemukan', 'paragraphs': [[['Jakarta', ',', 'CNN', 'Indonesia', '-', '-', 'Menteri', 'Koordinator', 'bidang', 'Politik', ',', 'Hukum', ',', 'dan', 'Keamanan', '(', 'Menko', 'Polhukam', ')', 'Wiranto', 'yakin', 'uang', 'jemaah', 'yang', 'dikumpulkan', 'bos', 'First', 'Travel', ',', 'Andika', 'Surachman', 'dan', 'Anniesa', 'Hasibuan', ',', 'bisa', 'ditemukan', '.']], [['"', 'Uang', 'itu', 'kan', 'cuma', 'pindah', 'tempat', ',', 'enggak', 'ada', 'uang', 'menguap', ',', 'itu', 'enggak', 'ada', '.'], ['Enggak', 'ada', 'tiba-tiba', 'uang', 'hilang', 'menguap', ',', 'mesti', 'pindah', 'tempat', '.'], ['Kita', 'cari', 'itu', ',', '"', 'ujar', 'Wiranto', 'di', 'Hotel', 'Arya', 'Duta', ',', 'Jakarta', ',', 'Rabu', '\xa0', '(', '30', '/', '

In [3]:
def join_data(dataset):
    documents = []
    summaries = []
    for item in dataset:
        # Menggabungkan kalimat dalam paragraf menjadi satu string
        document = [" ".join(sent) for para in item["paragraphs"] for sent in para]
        document = " ".join(document)  # Gabungkan semua paragraf menjadi satu string

        # Menggabungkan ringkasan
        summary = " ".join([" ".join(s) for s in item["summary"]])  # Gabungkan semua elemen dalam summary

        documents.append(document)
        summaries.append(summary)

    return documents, summaries

# Memproses data
train_docs, train_summaries = join_data(train_data)

In [4]:
print(train_docs[0])
print(train_summaries[0])
len(train_docs)

Jakarta , CNN Indonesia - - Menteri Koordinator bidang Politik , Hukum , dan Keamanan ( Menko Polhukam ) Wiranto yakin uang jemaah yang dikumpulkan bos First Travel , Andika Surachman dan Anniesa Hasibuan , bisa ditemukan . " Uang itu kan cuma pindah tempat , enggak ada uang menguap , itu enggak ada . Enggak ada tiba-tiba uang hilang menguap , mesti pindah tempat . Kita cari itu , " ujar Wiranto di Hotel Arya Duta , Jakarta , Rabu   ( 30 / 8 ) . Pernyataan Wiranto itu merujuk tipisnya jumlah dana yang berada di rekening bos First Travel , sementara jemaah yang dana keberangkatan umrahnya ditilap mencapai sekitar Rp 848 miliar . Wiranto mengatakan , uang yang diambil bos First Travel tersebut penting untuk ditemukan karena harus dikembalikan kepada para jemaah yang telah ditipu . Mantan Panglima ABRI itu juga menyampaikan , First Travel tetap menjadi pihak yang harus bertanggung jawab untuk mengembalikan uang para jemaah . " Yang gantinya siapa yang merugikan rakyat , lagi dilihat asetn

3752

In [5]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# test
train_docs, test_docs, train_summaries, test_summaries = train_test_split(
    train_docs, train_summaries, test_size=0.05, random_state=42
)

print(f"Train: {len(train_docs)}")
print(f"Test: {len(test_docs)}")

Train: 3564
Test: 188


In [6]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, GenerationConfig

# Memuat tokenizer dan model
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# model_name='google/flan-t5-base'

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for index in example_indices_full:
    document = train_docs[index]
    summary = train_summaries[index]
    prompt += f"""

Example Text: {document}

Example Summary: {summary}
    """
  document = train_docs[example_index_to_summarize]
  prompt += f"""

Text: {document}

Summary:
    """
  return prompt

generation_config = GenerationConfig(max_new_tokens = 100, do_sample = True, temperature = 0.1)

In [8]:
# example_indices_full = [10]
# example_index_to_summarize = 50

# one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

# print(one_shot_prompt)

In [9]:
# summary = train_summaries[example_index_to_summarize]

# inputs = tokenizer(one_shot_prompt, return_tensors='pt', truncation=True)
# output = tokenizer.decode(
#     model.generate(
#         inputs["input_ids"],
#         generation_config=generation_config,
#     )[0],
#     skip_special_tokens=True
# )

# print(f'Reference Summary:\n{summary}\n')
# print(f'One-Shot Generated Summnary:\n{output}')

In [10]:
# example_indices_full = [10, 60, 110, 160, 210]
# example_index_to_summarize = 50

# few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

# print(few_shot_prompt)

In [11]:
# summary = train_summaries[example_index_to_summarize]

# inputs = tokenizer(few_shot_prompt, return_tensors='pt', truncation=True)
# output = tokenizer.decode(
#     model.generate(
#         inputs["input_ids"],
#         generation_config=generation_config,
#     )[0],
#     skip_special_tokens=True
# )

# print(f'Reference Summary:\n{summary}\n')
# print(f'Few-Shot Generated Summnary:\n{output}')

In [12]:
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# Daftar untuk menyimpan hasil ringkasan dan referensi
generated_summaries = []
reference_summaries = []

# Menggunakan dataset pengujian
for example_index_to_summarize in range(10):
    # Mengambil beberapa contoh untuk prompt
    example_indices_full = [10]
    one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

    # Menghasilkan ringkasan
    generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
    inputs = tokenizer(one_shot_prompt, return_tensors='pt', truncation=True)

    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            generation_config=generation_config,
        )[0],
        skip_special_tokens=True
    )

    # Menyimpan ringkasan yang dihasilkan dan referensi
    generated_summaries.append(output)
    reference_summaries.append(test_summaries[example_index_to_summarize])

# Menghitung skor BLEU untuk setiap ringkasan yang dihasilkan
bleu_scores = []
for generated, reference in zip(generated_summaries, reference_summaries):
    reference_tokens = reference.split()  # Tokenisasi referensi
    generated_tokens = generated.split()  # Tokenisasi hasil
    score = sentence_bleu([reference_tokens], generated_tokens)  # Menghitung skor BLEU
    bleu_scores.append(score)

# Menampilkan hasil
for i in range(10):
    print(f"Reference Summary {i + 1}:\n{reference_summaries[i]}\n")
    print(f"Generated Summary {i + 1}:\n{generated_summaries[i]}\n")
    print(f"BLEU Score {i + 1}: {bleu_scores[i]}\n")

# Rata-rata skor BLEU
average_bleu_score = np.mean(bleu_scores)
print(f"Average BLEU Score: {average_bleu_score:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Reference Summary 1:
Penyelidik AS dilaporkan sudah bertemu dengan Christopher Steele , mantan mata-mata Inggris yang membocorkan isu hubungan antara tim kampanye Donald Trump dan Rusia pada masa pemilihan umum 2016 . Pertemuan antara penyelidik dari FBI , Robert Mueller , bertemu dengan Steele pada musim panas lalu . Nama Steele menjadi perhatian publik setelah merilis berkas penelitian politik yang menguak kemungkinan intervensi Rusia untuk memenangkan Trump .

Generated Summary 1:
<extra_id_0> , ” tambahnya .

BLEU Score 1: 8.90203994183703e-237

Reference Summary 2:
Indosat Ooredoo telah memperkenalkan kepada publik produk baru mereka berupa   platform Internet of Things   ( IoT ) yang mereka namakan " NEXThing " . Indosat berupaya menyentuh aspek bisnis yang tergolong masih jarang di Indonesia . Teknologi IoT sendiri adalah kunci sentral yang menjembatani kebutuhan digital perusahaan dalam mengelola aset , analisis tren , dan yang paling penting pengumpulan data dari berbagai pera

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [13]:
# Daftar untuk menyimpan hasil ringkasan dan referensi
generated_summaries = []
reference_summaries = []

# Menggunakan dataset pengujian
for example_index_to_summarize in range(10):
    # Mengambil beberapa contoh untuk prompt
    example_indices_full = [10, 60, 110, 160, 210]
    few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

    # Menghasilkan ringkasan
    generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
    inputs = tokenizer(few_shot_prompt, return_tensors='pt', truncation=True)

    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            generation_config=generation_config,
        )[0],
        skip_special_tokens=True
    )

    # Menyimpan ringkasan yang dihasilkan dan referensi
    generated_summaries.append(output)
    reference_summaries.append(test_summaries[example_index_to_summarize])

# Menghitung skor BLEU untuk setiap ringkasan yang dihasilkan
bleu_scores = []
for generated, reference in zip(generated_summaries, reference_summaries):
    reference_tokens = reference.split()  # Tokenisasi referensi
    generated_tokens = generated.split()  # Tokenisasi hasil
    score = sentence_bleu([reference_tokens], generated_tokens)  # Menghitung skor BLEU
    bleu_scores.append(score)

# Menampilkan hasil
for i in range(10):
    print(f"Reference Summary {i + 1}:\n{reference_summaries[i]}\n")
    print(f"Generated Summary {i + 1}:\n{generated_summaries[i]}\n")
    print(f"BLEU Score {i + 1}: {bleu_scores[i]}\n")

# Rata-rata skor BLEU
average_bleu_score = np.mean(bleu_scores)
print(f"Average BLEU Score: {average_bleu_score}")

Reference Summary 1:
Penyelidik AS dilaporkan sudah bertemu dengan Christopher Steele , mantan mata-mata Inggris yang membocorkan isu hubungan antara tim kampanye Donald Trump dan Rusia pada masa pemilihan umum 2016 . Pertemuan antara penyelidik dari FBI , Robert Mueller , bertemu dengan Steele pada musim panas lalu . Nama Steele menjadi perhatian publik setelah merilis berkas penelitian politik yang menguak kemungkinan intervensi Rusia untuk memenangkan Trump .

Generated Summary 1:
<extra_id_0> . ” tambahnya . Contoh : Contoh : Contoh : Contoh : .

BLEU Score 1: 3.24476966238448e-233

Reference Summary 2:
Indosat Ooredoo telah memperkenalkan kepada publik produk baru mereka berupa   platform Internet of Things   ( IoT ) yang mereka namakan " NEXThing " . Indosat berupaya menyentuh aspek bisnis yang tergolong masih jarang di Indonesia . Teknologi IoT sendiri adalah kunci sentral yang menjembatani kebutuhan digital perusahaan dalam mengelola aset , analisis tren , dan yang paling penti